Check directory

In [2]:
import os
%pwd

'e:\\RenalTumor\\research'

Change directory

In [3]:
os.chdir('../')
%pwd

'e:\\RenalTumor'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from src.KidneyDiseaseClassifier.constants import *
from src.KidneyDiseaseClassifier.utils.common import read_yaml,create_directories


In [6]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH
    ):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        create_directories([self.config.artifacts_root]) #artifacts

    def get_data_ingestion_config(self)->DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir, #artifacts/data_ingestion
            source_URL=config.source_URL,
            local_data_file=config.local_data_file, #artifacts/data_ingestion/data.zip
            unzip_dir=config.unzip_dir #artifacts/data_ingestion
        )


        return data_ingestion_config

In [7]:
import os 
import zipfile
import gdown
from src.KidneyDiseaseClassifier import logger


In [8]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config

    def download_file(self)->str:
        '''Fetch data from url'''
        try:
            dataset_url=self.config.source_URL
            zip_download_dir=self.config.local_data_file
            os.makedirs('artifacts/data_ingestion',exist_ok=True)
            logger.info(f'Downloading data from {dataset_url} into file{zip_download_dir}')
            file_id=dataset_url.split('/')[-2]
            prefix='https://drive.google.com/uc?/export=download&id='+file_id

            gdown.download(prefix,zip_download_dir)
            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            logger.error('Error !! ',e)
            raise(e)
        
    def extract_zip_file(self):
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [ ]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.error(f"ERROR !! {e}")
    raise e

[Wed, 02 Jul 2025 23:52:18: INFO: common: Loading Yaml file succesfully.]
[Wed, 02 Jul 2025 23:52:18: INFO: common: Loading Yaml file succesfully.]
[Wed, 02 Jul 2025 23:52:18: INFO: common: Directory created at: artifacts]
[Wed, 02 Jul 2025 23:52:18: INFO: common: Directory created at: artifacts/data_ingestion]
[Wed, 02 Jul 2025 23:52:18: INFO: 1442355725: Downloading data from https://drive.google.com/file/d/1vAoKHay3vqy4VWNcr0sg7Ohgg2z-nGX7/view?usp=sharing into fileartifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1vAoKHay3vqy4VWNcr0sg7Ohgg2z-nGX7
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1vAoKHay3vqy4VWNcr0sg7Ohgg2z-nGX7&confirm=t&uuid=6ebff21c-2428-4599-be81-5f53304b4bd0
To: e:\RenalTumor\artifacts\data_ingestion\data.zip
  1%|          | 10.5M/1.63G [01:20<4:53:22, 91.9kB/s]